In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import random

dataRep = '../data/'
scriptRep = '../script/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar

In [32]:
testFrac = 0.2

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')

raw = np.load('../data/dataset/test2.npz')
image = np.expand_dims(raw['image'], 3)
charOutput = raw['characterClass']

index = np.arange(image.shape[0])
np.random.shuffle(index)
image = image[index]/255.0
charOutput = charOutput[index]

nFrac = int(image.shape[0]*(1.-testFrac))
print(image.shape[0], image.shape[1:], nFrac)

464 (32, 32, 1) 371


In [33]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_20 (Spatia (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_21 (Spatia (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_22 (Spatia (None, 12, 12, 32)       

In [34]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=10)

Epoch 1/10
371/371 [==============================] - 3s 7ms/sample - loss: 8.2142 - acc: 0.0081
Epoch 2/10
371/371 [==============================] - 2s 6ms/sample - loss: 6.8916 - acc: 0.0296
Epoch 3/10
371/371 [==============================] - 2s 6ms/sample - loss: 5.9409 - acc: 0.0243
Epoch 4/10
371/371 [==============================] - 2s 5ms/sample - loss: 5.4742 - acc: 0.0377
Epoch 5/10
371/371 [==============================] - 2s 6ms/sample - loss: 5.2227 - acc: 0.0485
Epoch 6/10
371/371 [==============================] - 2s 6ms/sample - loss: 4.9819 - acc: 0.0323
Epoch 7/10
371/371 [==============================] - 2s 6ms/sample - loss: 4.8819 - acc: 0.0377
Epoch 8/10
371/371 [==============================] - 2s 6ms/sample - loss: 4.9395 - acc: 0.0350
Epoch 9/10
371/371 [==============================] - 2s 6ms/sample - loss: 4.8519 - acc: 0.0323
Epoch 10/10
371/371 [==============================] - 2s 6ms/sample - loss: 4.8242 - acc: 0.0377


In [30]:
charOutput[:nFrac].shape

(371, 4787, 1)

In [31]:
model.output_shape

(None, 4787)

In [19]:
index

array([598418, 466034, 531642, ..., 604534, 292986, 552445])

In [23]:
np.max(image)

255